<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/openai/FunctionCalling_RealExample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade openai
! pip install python-dotenv
! pip install --upgrade langchain

! pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
a = load_dotenv('drive/MyDrive/env')
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from typing import List
from pydantic import BaseModel, Field

In [ ]:
class ClosingBalanceSearch(BaseModel):
    """Call this with a year to get the closing balance for that year"""
    year: str = Field(description="year to get closing balance for")

In [ ]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
closingbalancesearchfunction = convert_pydantic_to_openai_function(ClosingBalanceSearch)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [ ]:
closingbalancesearchfunction

{'name': 'ClosingBalanceSearch',
 'description': 'Call this with a year to get the closing balace for that year',
 'parameters': {'properties': {'year': {'description': 'year to get closing balance for',
    'type': 'string'}},
  'required': ['year'],
  'type': 'object'}}

In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=openai.api_key)

In [ ]:
chat.invoke("I want to know the closing balance for the year 1998", functions=[closingbalancesearchfunction])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"year":"1998"}', 'name': 'ClosingBalanceSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 76, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-b24a0dee-aa8b-4a23-ad65-f38966d9f544-0')

In [ ]:
model_with_function = chat.bind(functions=[closingbalancesearchfunction])

In [ ]:
model_with_function.invoke("I want to know the closing balance for the year 1998")

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
# from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
import json

In [ ]:
parameterExtractionChain = prompt | model_with_function #| JsonOutputFunctionsParser()
r = parameterExtractionChain.invoke({"input": "I want to know the closing balance for the year 1998"})
r.additional_kwargs['function_call']['name']

'ClosingBalanceSearch'

In [ ]:
json.loads(r.additional_kwargs['function_call']['arguments'])['year']

'1998'